# OpenTripPlanner 


__We can set our own route planner for a given OSM map and GTFS file and query server on localhost for a detailed routes__


### OTP walkthrough

from: http://docs.opentripplanner.org/en/latest/Basic-Tutorial/

1. Install Java SDK
2. Download [latest](https://repo1.maven.org/maven2/org/opentripplanner/otp/1.4.0/otp-1.4.0-shaded.jar) .jar from [Maven](https://repo1.maven.org/maven2/org/opentripplanner/otp/1.4.0/)
3. Download gtfs.zip of a given city into a `folder` from `transit.land`
4. Download osm.pgf of your area from [here](https://www.interline.io/osm/extracts/) (need to login to get API token `7b7b8199-2fed-4e24-840c-5f73cba835a4`) - all 6 cities are there
5. call from terminal `java -Xmx6G -jar otp-1.4.0-shaded.jar --build /Users/rkucharski/Documents/otp --inMemory` changing `6G` as memory allocated (6GB) and `/Users/rkucharski/Documents/otp` to your path
6. if succesful you shall be able to open a web search in your browser: `http://localhost:8080/`
7. and query it via API, for instance: 'http://localhost:8080/otp/routers/default/plan?fromPlace=52.25177%2C20.92758&toPlace=52.20550%2C21.00517&time=1%3A02pm&date=04-09-2020&mode=TRANSIT%2CWALK&maxWalkDistance=5000&arriveBy=false'


In [1]:
import requests
import folium
from shapely.geometry import Point
from datetime import datetime
import polyline
import pandas as pd
import geopandas as gpd
import sys
from IPython.display import Markdown, display
import json

In [2]:
sys.path.append('/Users/rkucharski/Documents/GitHub/pt_accessibility/src/')

In [3]:
from server20 import *

In [4]:
df = pd.read_csv('../data/otp/DC/DC.csv')

In [10]:
row = df.sample(1).squeeze()
query = make_query(row)
r = requests.get("http://localhost:8080/otp/routers/default/plan", params=query)
route = parse_OTP_response(r.json())
plan = r.['plan']
plot(plan, color = 'green')

TypeError: 'Response' object is not subscriptable

## Sample query

[query synthax](http://dev.opentripplanner.org/apidoc/1.0.0/resource_PlannerResource.html)  
[response json structure](http://dev.opentripplanner.org/apidoc/1.0.0/json_Response.html)

In [8]:
route = query_PT(fromPlace = "52.25177,20.92758", toPlace = "52.20550,21.00517", time = "10:02am")
plan = route['plan']

NameError: name 'query_PT' is not defined

In [6]:
with open('plan.json', 'w') as outfile:
    json.dump(route, outfile)

In [7]:
plot(plan, color = 'green')

Trip from (20.9276,52.2518) to (21.0052,52.2055) at 1586419320000. 
3 connections found. 
Best one is 56min (624m walk, 1 transfer(s), wait time 2.45min)

LEG 	 MODE 	DIST 	TIME
-----------------------------
1	WALK	282	235
2	BUS	3123	600
3	WALK	123	95
4	TRAM	10000	2100
5	WALK	219	181


#### Query times: 15s per 100 queries

In [9]:
fromPlace = "52.25177,20.92758" 
toPlace = "52.20550,21.00517"
time = "11:02pm"
"""
Queries the server (with a given GTFS and OSM loaded) for a route 'fromPlace' 'toPlace' at 'time', Returns .json response
"""     
query = dict() # full query: http://dev.opentripplanner.org/apidoc/1.0.0/resource_PlannerResource.html
query['fromPlace'] = fromPlace
query['toPlace'] = toPlace
query['time'] = time
query['date'] = "04-09-2020"  # !!!!!!!!! MM-DD-YYYY !!!!!!!!! (Americans...)
query['mode'] = "TRANSIT,WALK"
query['maxWalkDistance']=500
query['arriveBy']='false'

In [10]:
def just_query(query):
    
    r = requests.get("http://localhost:8080/otp/routers/default/plan", params = query)
    route = r.json()  
    return route

In [11]:
just_query(query)

{'requestParameters': {'date': '04-09-2020',
  'mode': 'TRANSIT,WALK',
  'arriveBy': 'false',
  'fromPlace': '52.25177,20.92758',
  'toPlace': '52.20550,21.00517',
  'time': '11:02pm',
  'maxWalkDistance': '500'},
 'error': {'id': 406,
  'msg': 'No transit times available. The date may be past or too far in the future or there may not be transit service for your trip at the time you chose.',
  'message': 'NO_TRANSIT_TIMES',
  'noPath': True},
 'debugOutput': {'precalculationTime': 0,
  'pathCalculationTime': 0,
  'pathTimes': [],
  'renderingTime': 0,
  'totalTime': 0,
  'timedOut': False},
 'elevationMetadata': {'ellipsoidToGeoidDifference': -33.728873142311954,
  'geoidElevation': False}}

In [41]:
%timeit just_query(query)

134 ms ± 3.58 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [38]:
%timeit [query_PT() for _ in range(100)]

14.1 s ± 375 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


---

In [16]:
def make_query(row):
    """
    creates OTP query from the single row in uber data
    """
    query = dict()
    query['fromPlace'] = "{},{}".format(row.origin_lat, row.origin_lng)
    query['toPlace'] = "{},{}".format(row.dest_lat, row.dest_lng)
    hour, minute, _ = row.start_time.split(' ')[1].split(':')
    if int(hour) < 12:
        ampm = 'am'
    else:
        hour = int(hour) - 12
        ampm = 'pm'
    query['time'] = "{:02d}:{}{}".format(int(hour), minute, ampm)

    year, month, day = row.start_time.split(' ')[0].split('-')
    query['date'] = "{}-{}-{}".format(month, day, year)

    query['mode'] = "TRANSIT,WALK"
    query['maxWalkDistance'] = 2000
    query['arriveBy'] = 'false'
    return query

In [3]:
def query_PT(URL = "http://localhost:8080/otp/routers/default/plan", fromPlace = "52.25177,20.92758", toPlace = "52.20550,21.00517", time = "11:02pm"):
    """
    Queries the server (with a given GTFS and OSM loaded) for a route 'fromPlace' 'toPlace' at 'time', Returns .json response
    """     
    query = dict() # full query: http://dev.opentripplanner.org/apidoc/1.0.0/resource_PlannerResource.html
    query['fromPlace'] = fromPlace
    query['toPlace'] = toPlace
    query['time'] = time
    query['date'] = "04-09-2020"  # !!!!!!!!! MM-DD-YYYY !!!!!!!!! (Americans...)
    query['mode'] = "TRANSIT,WALK"
    query['maxWalkDistance']=500
    query['arriveBy']='false'
    r = requests.get(URL, params = query)
    route = r.json()    
    return route

In [8]:
def plot(plan, color = 'green'):
    """
    Plots the returned path in readable formand plots it on the map"""
    ret_str = "Trip from ({:.4f},{:.4f}) to ({:.4f},{:.4f}) at {}. \n{} connections found. \nBest one is {:.0f}min ({:.0f}m walk, {} transfer(s), wait time {:.2f}min)".format(plan['from']['lon'], plan['from']['lat'], 
          plan['to']['lon'], plan['to']['lat'], 
          plan['date'],
          len(plan['itineraries']), 
          plan['itineraries'][0]['duration']/60, 
          plan['itineraries'][0]['walkDistance'], 
          plan['itineraries'][0]['transfers'], 
          plan['itineraries'][0]['waitingTime']/60 )
    
    print(ret_str)
    print()
    print("LEG \t MODE \tDIST \tTIME")
    print('-----------------------------')
    i=1
    for leg in plan['itineraries'][0]['legs']:
        print("{}\t{}\t{:.0f}\t{:.0f}".format(i,leg['mode'], leg['distance'], leg['duration']))
        i+=1
        
    ret = list()
    tile = 'Stamen Toner' # 'StamenTerrain'
    tile = "https://tile.thunderforest.com/transport/{z}/{x}/{y}.png?apikey=54d9f38859864044ae1906a121f1e942"
    markers = list()
    for leg in plan['itineraries'][0]['legs']:
        decoded = polyline.decode(leg['legGeometry']['points'])
        ret += decoded
        markers.append(decoded[0])
    points = folium.PolyLine(locations = ret, color=color, weight=5, opacity=0.6)
    
    #points = folium.features.GeoJson()
    CENTER = ((plan['from']['lat']+plan['to']['lat'])/2,(plan['from']['lon']+plan['to']['lon'])/2)
    map_osm = folium.Map(location=CENTER,
                             zoom_start=13,tiles=tile,attr="toner-bcg", control_scale=True)
    for marker in markers:
        folium.CircleMarker(marker, radius = 5, color = color, opacity = 0.7).add_to(map_osm)
    map_osm.add_child(points)
    return map_osm

---